# Testing how to implement Piecewise Layers

## TODO:
1. Figure out how to do batching with this code
2. Ensure this is the most efficient
3. Polish up the mess

In [2]:
import numpy as np
import tensorflow as tf

In [3]:
# Use eager execution to make debugging easier (causes tensorflow to evaluate functions immediately)
tf.enable_eager_execution()

## Piecewise Linear

In [71]:
# Create bin array with nbins, each with a width of 1.0/nbins. 
# Note: the bins array contains the bin edges including 0, which makes finding alpha easier later on
nbins = 32
width = 1.0/nbins
bins = np.array([[x*width for x in range(nbins+1)] for i in range(3)])

# Print bins for testing purposes
bins

array([[0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.96875, 1.     ],
       [0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.96875, 1.     ],
       [0.     , 0.03125, 0.0625 , 0.09375, 0.125  , 0.15625, 0.1875 ,
        0.21875, 0.25   , 0.28125, 0.3125 , 0.34375, 0.375  , 0.40625,
        0.4375 , 0.46875, 0.5    , 0.53125, 0.5625 , 0.59375, 0.625  ,
        0.65625, 0.6875 , 0.71875, 0.75   , 0.78125, 0.8125 , 0.84375,
        0.875  , 0.90625, 0.9375 , 0.968

In [5]:
# randomly assign unnormalized values to piecewise PDF, here we will be using a total of 6 dimensions (|B|=3)
Q = np.random.rand(3,nbins)

# normalize the piecewise PDF such that the integral is 1
total = np.sum(Q,axis=1)
Q[0]=Q[0]/total[0]
Q[1]=Q[1]/total[1]
Q[2]=Q[2]/total[2]

# Insert a 0 to the beginning to match the size to the bin array
Q = np.insert(Q,0,0,axis=1)

# Print out values of Q for testing purposes
Q

In [9]:
# Generate a single phase space point in xB
xB = np.random.rand(1,3)

# Print out xB for testing purposes
xB

In [18]:
# searchsorted finds the location that each xB should be inserted into bins in order to keep the bins array sorted
# The transpose is needed to make sure that each point is compared to the right dimension
b=tf.searchsorted(bins,tf.transpose(xB),side='right')

# The result returns a transpose of a bin location array, take the transpose and print for testing
tf.transpose(b)

In [21]:
# Test to ensure that the correct Q values are pulled out from the arrays
# Note: that each dimension pulls out each bin, there may be a way around this, 
#       but the desired result is on the diagonal
Q[:,b[:,0]]

array([[0.01420116, 0.00373426, 0.00795862],
       [0.00482652, 0.05512942, 0.02111121],
       [0.03036283, 0.05347045, 0.03404066]])

In [22]:
# This calculates the PDF for the Piecewise Linear layer, 
# it takes the result from above and takes the product of the diagonal.
# Again there may be a better way to do this
tf.reduce_prod(tf.diag_part(Q[:,b[:,0]]))

<tf.Tensor: id=123, shape=(), dtype=float64, numpy=2.6650502460893233e-05>

In [27]:
# Calculate the alpha value in each dimension. 
# Note: the -1 in the bins term, this is to use the left edge of the bin instead of the right edge
alpha = (xB-tf.diag_part(bins[:,b[:,0]-1]))/width

# Print out the values of alpha for testing
alpha

In [58]:
# Calculates the first part of the coupling layer term (\alpha Q_{ib}) in each dimension
tf.diag_part(alpha*Q[:,b[:,0]])

<tf.Tensor: id=320, shape=(3,), dtype=float64, numpy=array([0.01016649, 0.0007496 , 0.02742373])>

In [66]:
# Calculates the second part of the coupling layer term (sum_{k=1}^{b-1} Q_{ik}) in each dimension
# There may be a more efficient way of doing this.
tf.diag_part(np.cumsum(Q,axis=1)[:,b[:,0]])

<tf.Tensor: id=371, shape=(3,), dtype=float64, numpy=array([0.01420116, 0.38524313, 0.52350583])>

In [69]:
# Calculates the full coupling layer terms
tf.diag_part(alpha*Q[:,b[:,0]])+tf.diag_part(np.cumsum(Q,axis=1)[:,b[:,0]])

<tf.Tensor: id=408, shape=(3,), dtype=float64, numpy=array([0.02436766, 0.38599273, 0.55092956])>